# Úkol č. 2 - předzpracování dat a binární klasifikace (do 2. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [220]:
import math
import pandas as pd
import numpy as np

In [221]:
df = pd.read_csv('data.csv')

Drop unimportant columns

In [222]:
display(df.head())
display(df.columns)
df.drop(columns=['ID', 'name', 'ticket', 'cabin', 'home.dest'], inplace=True)
display(df.head())

,ID,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,home.dest
0,0,1,3,"Dorking, Mr. Edward Arthur",male,19.0,0,0,A/5. 10482,8.0500,NaN,S,"England Oglesby, IL"
1,1,1,2,"Smith, Miss. Marion Elsie",female,40.0,0,0,31418,13.0000,NaN,S,NaN
2,2,0,3,"Hegarty, Miss. Hanora ""Nora""",female,18.0,0,0,365226,6.7500,NaN,Q,NaN
3,3,0,3,"Sage, Mr. John George",male,NaN,1,9,CA. 2343,69.5500,NaN,S,NaN
4,4,0,3,"Cacic, Miss. Marija",female,30.0,0,0,315084,8.6625,NaN,S,NaN


Index(['ID', 'survived', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked', 'home.dest'],
      dtype='object')

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,1,3,male,19.0,0,0,8.0500,S
1,1,2,female,40.0,0,0,13.0000,S
2,0,3,female,18.0,0,0,6.7500,Q
3,0,3,male,NaN,1,9,69.5500,S
4,0,3,female,30.0,0,0,8.6625,S


Convert categoric values to numeric representation

In [223]:
category_columns = ['embarked', 'sex']
df[category_columns] = df[category_columns].astype('category').apply(lambda x: x.cat.codes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  1000 non-null   int64  
 1   pclass    1000 non-null   int64  
 2   sex       1000 non-null   int8   
 3   age       797 non-null    float64
 4   sibsp     1000 non-null   int64  
 5   parch     1000 non-null   int64  
 6   fare      1000 non-null   float64
 7   embarked  1000 non-null   int8   
dtypes: float64(2), int64(4), int8(2)
memory usage: 49.0 KB


#### Solve NaN

Get distribution od age

In [224]:
df.age.max()
age_dist = {}
for i in range (0, 9):
    age_dist[i] = 0


Normalize

In [225]:
total = 0
t = df[df.age.notna()]
sum_check = 0
for a in t.age:
    age_dist[a//10] += 1
    total += 1
for age in age_dist:
    age_dist[age] /= total
    print(age, age_dist[age])
    sum_check += age_dist[age]
print(sum_check)

0 0.08531994981179424
1 0.13801756587202008
2 0.33877038895859474
3 0.22082810539523212
4 0.11166875784190715
5 0.07026348808030113
6 0.028858218318695106
7 0.005018820577164366
8 0.0012547051442910915
1.0


Distribute age to people with NaN age.

In [285]:
from numpy.random import choice


t = df[df.age.isnull()]
values = choice(list(age_dist.keys()), total, p = list(age_dist.values()))
it = 0
for i,row in t.iterrows():
    df.at[i,'age'] = values[it]
    it += 1


### Data are clear

In [228]:
df.isnull().sum(axis=0)

survived    0
pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64

In [229]:
from sklearn.tree import DecisionTreeClassifier
Xdata = df.iloc[:,1:]
ydata = df.iloc[:,0]
display(Xdata.head())
display(ydata.head())

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,1,19.0,0,0,8.0500,2
1,2,0,40.0,0,0,13.0000,2
2,3,0,18.0,0,0,6.7500,1
3,3,1,1.0,1,9,69.5500,2
4,3,0,30.0,0,0,8.6625,2


0    1
1    1
2    0
3    0
4    0
Name: survived, dtype: int64

### Split data

In [230]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import time


rd_seed = int(time.time())
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.25, random_state=rd_seed) 
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=rd_seed) 

Find best parameters

In [231]:
dt = DecisionTreeClassifier()
dt.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [232]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'max_depth': range(1,25), 
    'criterion': ['entropy', 'gini'],
    'splitter': ['best', 'random'],
    'min_samples': range(2,5)
}
param_comb = ParameterGrid(param_grid)

In [284]:
val_acc = []
train_acc = []
for params in param_comb:
    dt = DecisionTreeClassifier(max_depth=params['max_depth'],
    criterion=params['criterion'],splitter=params['splitter'],
    min_samples_split=params['min_samples'])
    dt.fit(Xtrain, ytrain)
    train_acc.append((metrics.accuracy_score(ytrain, dt.predict(Xtrain)), params))
    val_acc.append((metrics.accuracy_score(yval, dt.predict(Xval)), params))


In [236]:
best_val = val_acc[0][0]
best_param = val_acc[0][1]
for i in range(len(val_acc)):
    if best_val < val_acc[i][0]:
        best_val = val_acc[i][0]
        best_param = val_acc[i][1]
        print(best_val)

0.7978723404255319
0.824468085106383
0.8297872340425532


In [237]:
dt = DecisionTreeClassifier(max_depth=best_param['max_depth'],
    criterion=best_param['criterion'],splitter=best_param['splitter'],
    min_samples_split=best_param['min_samples'])
dt.fit(Xtrain, ytrain)

DecisionTreeClassifier(criterion='entropy', max_depth=5, splitter='random')

In [238]:
print('accuracy score (train): {0:.6f}'.format(metrics.accuracy_score(ytrain, dt.predict(Xtrain))))

accuracy score (train): 0.795374


In [239]:
print('accuracy score (test): {0:.6f}'.format(metrics.accuracy_score(ytest, dt.predict(Xtest))))

accuracy score (test): 0.740000


# EVALUATION

Proceed with same steps of preparation as for training:
 - prepare data
 - clean data
 - divide to parts
 - evaluate

Prepare data

In [262]:
df_eval = pd.read_csv('evaluation.csv')
df_eval.drop(columns=['ID', 'name', 'ticket', 'cabin', 'home.dest'], inplace=True)
category_columns = ['embarked', 'sex']
df_eval[category_columns] = df_eval[category_columns].astype('category').apply(lambda x: x.cat.codes)
display(df_eval)

,pclass,sex,age,sibsp,parch,fare,embarked
0,2,0,24.0,2,1,27.0000,2
1,2,0,25.0,1,1,30.0000,2
2,2,1,38.0,1,0,21.0000,2
3,3,0,19.0,1,0,16.1000,2
4,2,0,60.0,1,0,26.0000,2
...,...,...,...,...,...,...,...
304,3,0,18.5,0,0,7.2833,1
305,3,1,32.0,0,0,56.4958,2
306,3,1,22.5,0,0,7.2250,0
307,1,0,45.0,1,1,164.8667,2


Clean data 

In [263]:
m = int(df_eval.age.max())
age_dist = {}
for i in range (m):
    age_dist[i] = 0
total = 0
t = df_eval[df_eval.age.notna()]
sum_check = 0
for a in t.age:
    age_dist[a//10] += 1
    total += 1
for age in age_dist:
    age_dist[age] /= total
    sum_check += age_dist[age]
t = df_eval[df_eval.age.isnull()]
values = choice(list(age_dist.keys()), total, p = list(age_dist.values()))
it = 0
for i,row in t.iterrows():
    df_eval.at[i,'age'] = values[it]
    it += 1

In [264]:
# found missing item in fare column
t = df_eval[df_eval.fare.notna()]
median_list = []
for i, row in t.iterrows():
    median_list.append(row.fare)
median = np.median(median_list)
print(median)
t = df_eval[df_eval.fare.isnull()]
for i, row in t.iterrows():
    df_eval.at[i,'fare'] = median

14.75


Divide to parts

In [265]:
Xdata = df_eval.iloc[:,0:]

In [266]:
print(Xdata.isnull().sum(axis=0))

pclass      0
sex         0
age         0
sibsp       0
parch       0
fare        0
embarked    0
dtype: int64


Evaluate on best learned tree

In [286]:
# best_param = {'criterion': 'gini', 'max_depth': 5, 'min_samples': 4, 'splitter': 'random'}
df_out = pd.DataFrame()
df_out.insert(0,'survived',dt.predict(Xdata))
df_out.insert(0,'ID',range(1000,1309))
df_out.to_csv('results.csv',index=False)